In [1]:

from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from torchvision.models import inception_v3
from scipy.stats import entropy
from torchvision import transforms
from PIL import Image
import numpy as np
import torchvision.models as models
import warnings
warnings.filterwarnings("ignore")

# torch.cuda.empty_cache()
# torch.cuda.set_device(2)

def inception_score(imgs,device,batch_size=64, resize=True, splits=1):
    # Convert NumPy array to PyTorch tensor
    #imgs_tensor = torch.from_numpy(imgs)

    # Move the tensor to the desired device
    #imgs_tensor = imgs_tensor.to(device)
    imgs = imgs.to(device)
    #imgs = imgs_tensor
    N = imgs.size(0)
    if resize:
        imgs = nn.functional.interpolate(imgs, size=(299, 299), mode='bilinear', align_corners=False)
    #imgs = imgs.repeat(1, 3, 1, 1)  # Convert grayscale to RGB

    inception_model = models.inception_v3(pretrained=True, transform_input=False).eval().to(device)
    up = nn.Upsample(size=(299, 299), mode='bilinear', align_corners=False).to(device)

    scores = []
    for i in range(0, N, batch_size):
        batch = imgs[i:i + batch_size]
        batch = up(batch)
        logits = inception_model(batch)
        probs = torch.softmax(logits, dim=1)
        kl = probs * (torch.log(probs) - torch.log(torch.tensor(1.0 / probs.size(1)).to(device)))
        kl = kl.sum(1)
        scores.append(kl.cpu().numpy())

    scores = np.concatenate(scores)
    scores = np.exp(scores.mean())

    if splits > 1:
        kl = probs * (torch.log(probs) - torch.log(torch.tensor(1.0 / probs.size(1)).to(device)))
        kl = kl.sum(1).detach().cpu().numpy()
        split_scores = []
        for k in range(splits):
            part = kl[k * (N // splits): (k + 1) * (N // splits)]
            py = np.exp(part.mean())
            split_scores.append(py)
        return scores, np.std(split_scores)

    return scores

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

# Root directory for dataset
dataroot = "/home/stud1/Aman/dataset"


# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size =64

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 128

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 128

# Size of feature maps in discriminator
ndf = 128

# Number of training epochs
num_epochs = 1000

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

# We can use an image folder dataset the way we have it setup.
# Create the dataset
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize((128,128)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device("cuda:1" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Plot some training images
real_batch = next(iter(dataloader))
# plt.figure(figsize=(8,8))
# plt.axis("off")
# plt.title("Training Images")
# plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:32], padding=2, normalize=True).cpu(),(1,2,0)))
# plt.savefig('training_images_wgan.png')
# plt.close()
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.02.
netG.apply(weights_init)

# Print the model
#print(netG)

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, ndf * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf *16, 1, 4, 1, 0, bias=False),
            #nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))
    
# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
#print(netD)

# Initialize BCELoss function
criterion = nn.MSELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 0.9
fake_label = 0.1

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

# Commented out IPython magic to ensure Python compatibility.
# Training Loop
netD.load_state_dict(torch.load('discriminator_weights.pth'))
netG.load_state_dict(torch.load('generator_weights.pth'))

print('g done')


# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
i_score=[]
iters = 0
clip_value=0.01
fid_score_=[]

print("Starting Training Loop...")

for epoch in range(num_epochs):
    # Save generator weights
    torch.save(netG.state_dict(), 'generator_weights.pth')

    # Save discriminator weights
    torch.save(netD.state_dict(), 'discriminator_weights.pth')

    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
 
     # Clip discriminator parameters
        # for p in netD.parameters():
        #     p.data.clamp_(-clip_value, clip_value)

        # Reset gradients
        netD.zero_grad()

        # Train with real data
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label_real = torch.ones(b_size, 1).to(device)
        output_real = netD(real_cpu)
        errD_real = -torch.mean(output_real)
        D_x=output_real.mean().item()
        
        # Train with fake data
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label_fake = torch.zeros(b_size, 1).to(device)
        output_fake = netD(fake.detach())
        errD_fake = torch.mean(output_fake)
        D_G_z1 =output_fake.mean().item()

        # Backward and optimize
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        for p in netD.parameters():
            p.data.clamp_(-clip_value, clip_value)

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        # Reset gradients
        netG.zero_grad()

        # Generate fake data
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        #fake = netG(noise)

        # Compute discriminator output on fake data
        output = netD(fake)
        errG = -torch.mean(output)

        # Backward and optimize
        errG.backward()
        D_G_z2=output.mean().item()
        optimizerG.step()
        
        # Output training stats
        if i  == len(dataloader)-1:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f\tD(G(z)):  %.4f'
                   % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        
        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())
        
        # Check how the generator is doing by saving G's output on fixed_noise
        #if (iters % 39 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
        if i == len(dataloader)-1:
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
                vutils.save_image(fake, 'Check.png', normalize=True)
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
            
        iters += 1
    with torch.no_grad():
        fake = netG(fixed_noise).detach().cpu()
        fake_images = vutils.make_grid(fake, normalize=True).numpy()
        inception_score_value= inception_score(fake,device, batch_size=64, resize=True, splits=1)
        print("Num Epochs: {}, inception_score: {:.4f}".format(epoch,inception_score_value))
        i_score.append(inception_score_value)
        
        fid_score = calculate_fid(real_cpu, fake, batch_size, inception_batch_size=64, device=device)
        print("Num Epochs: {}, FID: {:.4f}".format(epoch,fid_score))
        fid_score_.append(fid_score)

        
    

    plt.figure(figsize=(10,5))
    plt.title("Generator and Discriminator Loss During Training")
    plt.plot(G_losses,label="G")
    plt.plot(D_losses,label="D")
    plt.xlabel("iterations")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.legend()
    file_name = 'Loss_b_{}_i_{}_e_{}_wgan.png'.format(batch_size, image_size, num_epochs)
    plt.savefig(file_name,dpi=300)
    plt.close()


    epochs_ = list(range(1, len(i_score) + 1))
    plt.figure(figsize=(10,5))
    plt.title('Inception Score over Epochs')
    plt.plot(epochs_, i_score, marker='o')  # 'o' for circular markers
    plt.xlabel('Epochs')
    plt.ylabel('Inception Score')
    plt.grid(True)
    #plt.legend()
    file_name = 'Inception_score_b_{}_i_{}_e_{}_wgan.png'.format(batch_size, image_size, num_epochs)
    plt.savefig(file_name,dpi=300)
    plt.close()

    with open('Inception_scores_b_{}_i_{}_e_{}_wgan.txt'.format(batch_size, image_size,num_epochs), 'w') as file:
        for score in i_score:
            file.write(str(score) + '\n')
            
    plt.figure(figsize=(10,5))
    plt.title('fid Score over Epochs')
    plt.plot(epochs_, fid_score_, marker='o')  # 'o' for circular markers
    plt.xlabel('Epochs')
    plt.ylabel('fid score')
    plt.grid(True)
    #plt.legend()
    file_name = 'fid_score_b_{}_i_{}_e_{}_wgan.png'.format(batch_size, image_size, num_epochs)
    plt.savefig(file_name,dpi=300)
    plt.close()

    with open('fid_score_b_{}_i_{}_e_{}_wgan.txt'.format(batch_size, image_size,num_epochs), 'w') as file:
        for score in fid_score_:
            file.write(str(score) + '\n')
            
#     fig = plt.figure(figsize=(12, 12))  # Increase the figure size to 12x12 inches
#     plt.axis("off")
#     ims = [[plt.imshow(np.transpose(i, (1, 2, 0)), animated=True)] for i in img_list]
#     ani = animation.ArtistAnimation(fig, ims, interval=200, repeat_delay=1000, blit=True)  # Decrease the interval to 200 milliseconds for faster animation
#     ani.save('b_{}_i_{}_wgane_e_{}.gif'.format(batch_size, image_size,epoch), writer='imagemagick', dpi=300)  # Include batch_size, image_size, num_epochs, and iterations in the filename, and increase the dpi to 300 for higher quality
#     plt.close()

    # fig = plt.figure(figsize=(8,8))
    # plt.axis("off")
    # ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
    # ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)
    # ani.save('b_128_i_128_wgan_10000.gif', writer='imagemagick')  # For GIF format
    # plt.close()

    # HTML(ani.to_jshtml())

    # Grab a batch of real images from the dataloader
    real_batch = next(iter(dataloader))

    # Plot the real images
    plt.figure(figsize=(45,45))
    plt.subplot(1,2,1)
    plt.axis("off")
    plt.title("Real Images")
    plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

    # Plot the fake images from the last epoch
    plt.subplot(1,2,2)
    plt.axis("off")
    plt.title("Fake Images")
    plt.imshow(np.transpose(img_list[-1],(1,2,0)))
    file_name = 'Real_vs_fake_b_{}_i_{}_e_{}_wgan.png'.format(batch_size, image_size,num_epochs)
    plt.savefig(file_name,dpi=300)
    plt.close()

Random Seed:  999
g done
Starting Training Loop...


Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [2]:
fixed_noise_ = torch.randn(100, nz, 1, 1, device=device)
imgs=fixed_noise
from torchmetrics.image.inception import InceptionScore
inception = InceptionScore()
# generate some images
imgs = torch.randint(0, 255, (100, 3, 299, 299), dtype=torch.uint8)
inception.update(imgs)
inception.compute()


# fake_ = netG(fixed_noise_).detach().cpu()
# #fake_images_ = vutils.make_grid(fake_, normalize=True).numpy()
# fake_images1000 = vutils.make_grid(fake_, normalize=True).numpy().transpose((1, 2, 0))  # Transpose to (H, W, C)
# vutils.save_image(fake_, 'fake1000.png', normalize=True)
# plt.imshow(fake_images1000)
# #plt.show(fake_images_\)
# a = netD(fixed_noise_).detach().cpu()

2023-04-15 12:32:41.030100: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-15 12:32:41.056711: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-15 12:32:41.441550: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /home/stud1/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth
100%|████████████████████████████████████

(tensor(1.0562), tensor(0.0167))

In [ ]:
import torch
from torch.nn.functional import adaptive_avg_pool2d
from scipy.linalg import sqrtm
import numpy as np



def calculate_fid(real_images, generated_images, batch_size, device, inception_batch_size=64):
    """
    Calculates Fréchet Inception Distance (FID) score for GAN-generated images.

    Args:
        real_images (torch.Tensor): Real images from the real dataset.
        generated_images (torch.Tensor): Generated images from the GAN model.
        batch_size (int): Batch size for calculating FID score.
        device (str): Device to run the calculations on ('cuda' or 'cpu').
        inception_model (torch.nn.Module): Pre-trained Inception model.
        inception_batch_size (int, optional): Batch size for calculating Inception features. Default is 64.

    Returns:
        float: FID score.
    """
    # Set models to evaluation mode
    inception_model = models.inception_v3(pretrained=True, transform_input=False).eval().to(device)
    inception_model.eval()

    # Move images to the specified device
    real_images = real_images.to(device)
    generated_images = generated_images.to(device)

    # Resize and normalize images
    real_images = adaptive_avg_pool2d(real_images, (299, 299))
    real_images = (real_images - 0.5) * 2.0
    generated_images = adaptive_avg_pool2d(generated_images, (299, 299))
    generated_images = (generated_images - 0.5) * 2.0

    # Calculate Inception features for real images
    real_features = []
    num_batches = real_images.size(0) // inception_batch_size
    for i in range(num_batches):
        batch = real_images[i * inception_batch_size:(i + 1) * inception_batch_size]
        real_features.append(inception_model(batch).detach().view(batch.size(0), -1))
    real_features = torch.cat(real_features, dim=0)

    # Calculate Inception features for generated images
    generated_features = []
    num_batches = generated_images.size(0) // inception_batch_size
    for i in range(num_batches):
        batch = generated_images[i * inception_batch_size:(i + 1) * inception_batch_size]
        generated_features.append(inception_model(batch).detach().view(batch.size(0), -1))
    generated_features = torch.cat(generated_features, dim=0)

#     # Calculate mean and covariance of real and generated features
#     real_mean = torch.mean(real_features, dim=0)
    # Move real features to CPU memory
    real_features = real_features.detach().cpu()

    # Calculate mean and covariance of real features
    real_mean = torch.mean(real_features, dim=0)
    real_cov = np.cov(real_features.T)
    
    generated_features = generated_features.detach().cpu()
    generated_mean = torch.mean(generated_features, dim=0)
    generated_cov = np.cov(generated_features.T)
    #print(real_cov)
    #print(generated_cov)

    # Calculate FID score
    mean_diff = real_mean - generated_mean
    cov_mean_sqrt = sqrtm(real_cov.dot(generated_cov)).real
    cov_mean_sqrt = torch.from_numpy(cov_mean_sqrt).to(device)
    # Convert real_cov and generated_cov to PyTorch tensors
    real_cov = torch.from_numpy(real_cov).to(device)
    generated_cov = torch.from_numpy(generated_cov).to(device)

    # Calculate FID score
    fid = torch.sqrt(torch.sum(mean_diff * mean_diff) + torch.trace(real_cov + generated_cov - 2 * cov_mean_sqrt))

    #fid = torch.sqrt(torch.sum(mean_diff * mean_diff) + torch.trace(real_cov + generated_cov - 2 * cov_mean_sqrt))

    return fid.item()


In [ ]:
import torch
from torchmetrics.image.inception import InceptionScore
inception = InceptionScore()
# generate some images
imgs = torch.randint(0, 255, (100, 3, 299, 299), dtype=torch.uint8)
inception.update(imgs)
inception.compute()


In [ ]:


# # Load real images from data loader

# real_images_iter = iter(dataloader)
# real_images = next(real_images_iter)
# real_images = real_images[0].to(device)

# # Generate images using trained generator
#   # Replace with your own generator
# netG.eval()  # Set to evaluation mode
# with torch.no_grad():
#     generated_images = netG(torch.randn(real_images.size(0), nz,1,1).to(device))
# generated_images = generated_images.cpu()  # Move to CPU if necessary

# # Calculate FID score
# fid_score = calculate_fid(real_images, generated_images,batch_size, inception_batch_size=64, device=device)

# print("FID score: {:.4f}".format(fid_score))
# fid_score


In [ ]:
total_batches = len(dataloader)
# for batch_idx, real_images in enumerate(dataloader):
#     for batch_idx in range(total_batches):
#     real_images = next(iter(dataloader))
#     real_images = real_images[0].to(device)
for batch_idx in range(total_batches):
    real_images = next(iter(dataloader))
    real_images = real_images[0].to(device)


    # Generate images using trained generator
    # Replace with your own generator
    netG.eval()  # Set to evaluation mode
    with torch.no_grad():
        generated_images = netG(torch.randn(batch_size, nz, 1, 1).to(device))  # Generate batch_size images
    generated_images = generated_images.cpu()  # Move to CPU if necessary

    # Calculate FID score
    fid_score = calculate_fid(real_images, generated_images, batch_size, inception_batch_size=64, device=device)

    print("Batch {}/{} - FID score: {:.4f}".format(batch_idx + 1, len(dataloader), fid_score))
